# Project Proposal (Matt)

The data we are interested in working with involves using February 2015 airline delay data and Twitter review data. The group members for this project will be Matt Kennedy, Natalie Myers, Myles Bridges, and Sedra Kurdi. This data can be located at https://www.kaggle.com/usdot/flight-delays and https://www.kaggle.com/crowdflower/twitter-airline-sentiment. The delay data is provided in CSV format, which will be imported into a Pandas data frame and cleaned by dropping columns missing large amounts of data and then rows with missing data. The Twitter data is provided in SQLite format, which will be imported to Python with SQL Alchemy and then cleaned in the same manner. Both datasets will then be transformed into many smaller data frames that will represent specific tables in the final database. These data frames will include tweet data for each unique airline, as well as data for cancelled, diverted, and delayed flights. The final data frames will be inserted into an SQL database and connected through a central table with a primary key of unique airline carrier code.

# Project Code

## Data Extraction

### Flight Data (Myles & Sedra)

MK NOTES
1. Flight Data - Columns to Drop: 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
2. Flight Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
3. Flight Data - Add Column for Total Delay (Add Together Departure & Arrival Delay Values)
4. Flight Data - Combine Date Value Columns into Single Date Column
5. Airline Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
6. Airport Data - Columns to Drop: 'LATITUDE', 'LONGITUDE'

In [ ]:
#ADD CODE HERE

### Tweet Data (Natalie)

MK NOTES
1. Columns to Drop: 'negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 'negativereason_gold', 'tweet_coord', 'tweet_location', 'user_timezone'
2. Convert Date Column to Only Include Date & Not Time
3. Convert Airline Name to Airline Code

In [ ]:
#ADD CODE HERE

## Data Transformation

### Flight Data (Myles & Sedra)

MK NOTES
1. Airport Data - No Changes
2. Airline Data - No Changes
3. Flight Data - Use Group By & Filtering to Create Individual Data Frames for Delayed/Early/Cancelled/Diverted Flights for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [ ]:
#ADD CODE HERE

### Tweet Data (Natalie)

MK NOTES
1. Use Group By & Filtering to Create Data Frames of Positive/Negative/Neutral Tweets for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [ ]:
#ADD CODE HERE

## Data Loading (Matt)

MK Notes
1. Create ERD for Completed Database
2. Create Database in PGAdmin
3. Use SQL Alchemy to Load Dataframes into Tables & Relate by Keys

In [ ]:
#ADD CODE HERE

# Project Analysis

## Data Extraction

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Transformation

### Flight Data (Myles & Sedra)

ADD TEXT HERE

### Tweet Data (Natalie)

ADD TEXT HERE

## Data Loading (Matt)

ADD TEXT HERE